<a href="https://colab.research.google.com/github/Jeffstr00/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-03 01:19:06--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  4.96MB/s    in 0.2s    

2022-04-03 01:19:07 (4.96 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
# Read in the review set from S3 as a DataFrame
from pyspark import SparkFiles
url = "https://jeffstr00-bucket.s3.amazonaws.com/amazon_reviews_us_Books_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

In [ ]:
# Check to see what type columns are in so that they will ultimately match with postgres schema
df.schema

StructType(List(StructField(marketplace,StringType,true),StructField(customer_id,IntegerType,true),StructField(review_id,StringType,true),StructField(product_id,StringType,true),StructField(product_parent,IntegerType,true),StructField(product_title,StringType,true),StructField(product_category,StringType,true),StructField(star_rating,StringType,true),StructField(helpful_votes,IntegerType,true),StructField(total_votes,IntegerType,true),StructField(vine,StringType,true),StructField(verified_purchase,StringType,true),StructField(review_headline,StringType,true),StructField(review_body,StringType,true),StructField(review_date,StringType,true)))

In [ ]:
# Change Star Rating column from string to integer
from pyspark.sql.types import IntegerType
df = df.withColumn("star_ratingTmp", df["star_rating"].cast(IntegerType())).drop("star_rating").withColumnRenamed("star_ratingTmp", "star_rating")
# Rearrange columns so that Star Rating is back in place
df = df.select("marketplace", "customer_id", "review_id", "product_id", "product_parent", "product_title", "product_category", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_headline", "review_body", "review_date")
df.schema

StructType(List(StructField(marketplace,StringType,true),StructField(customer_id,IntegerType,true),StructField(review_id,StringType,true),StructField(product_id,StringType,true),StructField(product_parent,IntegerType,true),StructField(product_title,StringType,true),StructField(product_category,StringType,true),StructField(star_rating,IntegerType,true),StructField(helpful_votes,IntegerType,true),StructField(total_votes,IntegerType,true),StructField(vine,StringType,true),StructField(verified_purchase,StringType,true),StructField(review_headline,StringType,true),StructField(review_body,StringType,true),StructField(review_date,StringType,true)))

### Create DataFrames to match tables

In [ ]:
# Import to_date function
from pyspark.sql.functions import to_date

In [ ]:
# Create the customers_table DataFrame
from pyspark.sql.functions import count
# customers_df = df.groupby("").agg({""}).withColumnRenamed("", "customer_count")
customers_df = df.groupby("customer_id").agg(count('customer_id')).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16747249|             4|
|     108460|             1|
|   11720077|            31|
|     437936|             1|
|     128131|             1|
|   39647402|             1|
|   13730111|             8|
|   43783459|             1|
|   12493167|           111|
|   40325486|            92|
|   44618558|             4|
|   17956020|             1|
|   10093406|             3|
|   40430762|             1|
|     134924|             1|
|   52427934|             1|
|   12879980|            15|
|   27680937|             7|
|   16186289|             1|
|   20326078|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# check to see how many rows are in Customers DF
customers_df.count()

4632497

In [ ]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
# products_df = df.select(["product_id", "product_title"]).drop_duplicates((["product_id"]))
# check to see how many products there are before dropping unique titles, to ensure that duplicates are being dropped
products_df = df.select(["product_id", "product_title"])
products_df.count()

10319090

In [ ]:
# drop duplicate Product IDs and then count
products_df = products_df.drop_duplicates((["product_id"]))
products_df.count()

2274543

In [ ]:
# show Products DF
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0001845357|    Ballad of Favour|
|0002168383|The Beatles: For ...|
|0002192365| Cage & Aviary Birds|
|0002216973|     Red Adam's Lady|
|0002230534|A single death: A...|
|0002234947|Miss Hobbema Pageant|
|0002261367|Unexpected Blessings|
|0006541151|Order Out of Chao...|
|0006544150|           In Xanadu|
|0006726372|I'll Go My Own Wa...|
|0006925227|The Vanishing Tre...|
|0007121040|Miss Marple's Fin...|
|0007155174|The Treasure Hunt...|
|000717201X|The Lord of the R...|
|0007178484|Reading "Lolita" ...|
|000718025X|Akbar and Jeff's ...|
|0007219040|The Secret Langua...|
|0007236093|  Dead Witch Walking|
|0007243626|The Yacoubian Bui...|
|0007255349|The Mobile Librar...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
products_df.tail(5)

[Row(product_id='B010EW8DZM', product_title='Schliemann of Troy: Treasure and Deceit'),
 Row(product_id='B010WNU2Z2', product_title='[ The Chase of the Golden Meteor (Bison Book) [ THE CHASE OF THE GOLDEN METEOR (BISON BOOK) ] By Verne, Jules ( Author )Oct-01-1998 Paperback By Verne, Jules ( Author ) Paperback 1998 ]'),
 Row(product_id='B011MEPJCG', product_title='Dead Wrong 2: Diana, Princess of Wales by Belzer, Richard, Wayne, David (2013) Paperback'),
 Row(product_id='B011T6XNNY', product_title='The Final Minute: (Tina Boyd 6) by Simon Kernick (15-Jan-2015) Hardcover'),
 Row(product_id='B011YTPMAI', product_title='Real Food for Rookies: Healthy Cooking - Traditional Food - Vibrant Health by Moeggenborg, Kelly (2015) Paperback')]

In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RJOVP071AVAJO|   25933450|0439873800|      84656342| 2015-08-31|
|R1ORGBETCDW3AI|    1801372|1623953553|     729938122| 2015-08-31|
| R7TNRFQAOUTX5|    5782091|142151981X|     678139048| 2015-08-31|
|R2GANXKDIFZ6OI|   32715830|014241543X|     712432151| 2015-08-31|
|R2NYB6C3R8LVN6|   14005703|1604600527|     800572372| 2015-08-31|
|R13U5PBJI1H94K|   36205738|0399170863|     559876774| 2015-08-31|
|R1H8UVH990F8VE|   44121167|1517007240|     299984591| 2015-08-31|
|R2MC0N30WZMRQ5|   16519255|0671728725|     821650353| 2015-08-31|
| R2NO2HXK16Y4J|   49361350|1111349533|     419457767| 2015-08-31|
|R245YIAVJK82ZL|   11423253|812211637X|     285887177| 2015-08-31|
| RFBMLBWQOZ1UM|   37536260|0679413359|      58983679| 2015-08-31|
| RHE1FEWV3KTQH|   23614950|0983990662|     222024699| 2015-08

In [ ]:
# count rows in Review ID DF
review_id_df.count()

10319090

In [ ]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RJOVP071AVAJO|          5|            0|          0|   N|                Y|
|R1ORGBETCDW3AI|          5|            0|          0|   N|                Y|
| R7TNRFQAOUTX5|          5|            0|          0|   N|                Y|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|                N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|                Y|
|R13U5PBJI1H94K|          2|            1|          1|   N|                N|
|R1H8UVH990F8VE|          5|            2|          2|   N|                N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|                Y|
| R2NO2HXK16Y4J|          5|            0|          0|   N|                Y|
|R245YIAVJK82ZL|          5|            0|          0|   N|     

In [ ]:
# count number of rows in Vine DF
vine_df.count()

10319090

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password:')
# Configure settings for RDS
mode = "append"

jdbc_url="jdbc:postgresql://datachallenge.c326vl9oo2i8.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# 8 mins
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)